# Creating IOC instances
## Background


## Overview
To generate an IOC instance of particular class you need four things:
- A podman image for the class of IOC you wish to create
- An `<ioc_class>.ibek.yaml` file (this should be located in the podman image for the IOC)
- An `<ioc_class>.schema.json` file
- An `<ioc_instance_name>.yaml` file


## Example (IOC for a PMAC motion controller)

### Creating an `<ioc-class>.schema.json`
To create a schema adhering YAML file, we need first to create is a schema for our class of IOC: `pmac.schema.json`

In [1]:
#Tutorial specific for this early stage. To be changed when ibek is completed
ibek=../ibek/BL45P-MO-IOC-01/ioc_boot.py
ioc_schema=/tmp/pmac.schema.json
ioc_class=../ibek/BL45P-MO-IOC-01/pmac.ibek.yaml

This is achieved by running the command:

In [2]:
python $ibek ioc-schema $ioc_schema $ioc_class

In [3]:
# Display the schema generated 
cat $ioc_schema

{
  "type": "object",
  "properties": {
    "ioc_name": {
      "type": "string",
      "description": "Name of the IOC"
    },
    "instances": {
      "type": "array",
      "items": {
        "anyOf": [
          {
            "type": "object",
            "properties": {
              "type": {
                "type": "string",
                "const": "pmac.PmacAsynIPPort",
                "default": "pmac.PmacAsynIPPort"
              },
              "name": {
                "type": "string",
                "description": "Name of the entity instance we are creating",
                "default": ""
              },
              "script": {
                "type": "array",
                "items": {
                  "type": "string",
                  "description": "scripts required for boot script"
                },
                "default": [
                  "pmacAsynIPConfigure({{name}}, {{IP + \"\" if \":\" in IP else IP + \":1025\"}})"
                ]
             

              },
              "ACCL": {
                "type": "number",
                "description": "Seconds to Velocity",
                "default": 0.5
              },
              "BDST": {
                "type": "number",
                "description": "BL Distance (EGU)"
              },
              "BVEL": {
                "type": "number",
                "description": "BL Velocity(EGU/s)"
              },
              "BACC": {
                "type": "string",
                "description": "BL Seconds to Veloc",
                "default": ""
              },
              "DHLM": {
                "type": "number",
                "description": "Dial High Limit"
              },
              "DLMM": {
                "type": "number",
                "description": "Dial low limit"
              },
              "HLM": {
                "type": "number",
                "description": "User High Limit"
              },
              "LLM": {
                "t

### Using a schema-aware editor to write an `<ioc-name>.ibek.yaml`
ibek users are of course free to use any schema-aware editor of their choice, however we have tested ibek with the `redhat.vscode-yaml` extension to Visual Studio Code and can present some of the functionality below.

![Title](images/yaml_extension.png)

To make use of a specific schema, users must associate their schema with their yaml file.

All `<ioc-name>.ibek.yaml` files have the same form. They have a name: `ioc-name` and a list of instances: `instances`:
![Title](images/blank_yaml.png)
Each instance will be of a pre-determined type for that class of IOC
![Title](images/creating_an_instance_of_specific_type.png)

The schema can be used to check whether a specific key is valid for a given instance type:
![Title](images/invalid_key_error_one.png)
![Title](images/invalid_key_error_two.png)

The schema can check whether an insatnce type is valid for this ioc class:
![Title](images/invalid_instance_type.png)
![Title](images/invalid_instance_type_two.png)

### Using ibek to generate an `ioc.boot` script
Once we have written our `<ioc-name>.ibek.yaml` file we can use ibek to generate the only instance specific part of our IOC, the `ioc.boot` script. Below is an example `<ioc-name>.ibek.yaml` file for an instance of a pmac class of IOC.

In [4]:
ioc_instance_name_yaml=../ibek/BL45P-MO-IOC-01/bl45p-mo-ioc-02.pmac.yaml
cat $ioc_instance_name_yaml

ioc_name: my_ioc
instances:
  - type: pmac.PmacAsynIPPort
    IP: 172.23.240.97
    name: BRICK1port

  - type: pmac.Geobrick
    name: BRICK1
    PORT: BRICK1port
    P: BRICK1
    IdlePoll: 200
    movingPoll: 100

  - type: pmac.DlsPmacAsynMotor
    PMAC: BRICK1
    P: MOTORBRICK1
    M: :motor1
    PORT: BRICK1
    ADDR: 1
    DESC: motor1
    MRES: 0.0001
    VELO: 20
    PREC: 3
    EGU: mm
    TWV: 1

  - type: pmac.DlsPmacAsynMotor
    PMAC: BRICK1
    P: MOTORBRICK1
    M: :motor2
    PORT: BRICK1
    ADDR: 2
    DESC: motor2
    MRES: 0.0001
    VELO: 20
    PREC: 3
    EGU: mm
    TWV: 1

  - type: pmac.DlsPmacAsynMotor
    PMAC: BRICK1
    P: MOTORBRICK1
    M: :motor3
    PORT: BRICK1
    ADDR: 3
    DESC: motor3
    MRES: 0.0001
    VELO: 20
    PREC: 3
    EGU: mm
    TWV: 1

  - type: pmac.DlsPmacAsynMotor
    PMAC: BRICK1
    P: MOTORBRICK1
    M: :motor4
    PORT: BRICK1
    ADDR: 4
    DESC: motor4
    MRES: 0.0001
    VELO: 20
    PREC: 3
    EGU: mm
    TWV: 1

  -

From this file we can use ibek to generate a boot script and helm chart using the command `python ibek build-ioc <ioc_instance_name>.yaml <ioc_class>.ibek.yaml`

In [5]:
python $ibek build-ioc $ioc_instance_name_yaml $ioc_class

iocs/bl45p-mo-ioc-02/config/ioc.boot
